Importing the Data

In [2]:
# Import Pandas and Datasets

import pandas as pd
cols = list(pd.read_csv("2019-2020.csv", nrows =1))
df18_19 = pd.read_csv('2018-2019.csv', usecols = [i for i in range(1, len(cols))])
df19_20 = pd.read_csv('2019-2020.csv', usecols = [i for i in range(1, len(cols))])
df20_21 = pd.read_csv('2020-2021.csv', usecols = [i for i in range(1, len(cols))])
df21_22 = pd.read_csv('2021-2022.csv', usecols = [i for i in range(1, len(cols))])
df22_23 = pd.read_csv('2022-2023.csv', usecols = [i for i in range(1, len(cols))])
dfx = [df18_19, df19_20, df20_21, df21_22, df22_23]

Handling Missing Values

In [3]:
# Create Function to Get Variable Names
def get_name(df_):
    name = [x for x in globals() if globals()[x] is df_][0]
    return name
    
# Check Datasets for Missing Values
for df in dfx:
    for col in df.columns:
        if df[col].isnull().values.any():
            print(get_name(df), col)

df18_19 SoT%
df18_19 G/SoT
df18_19 Avg. Distance of Shots
df18_19 Save%
df19_20 G/SoT
df19_20 Save%
df20_21 SoT%
df20_21 G/SoT
df20_21 Avg. Distance of Shots
df20_21 Save%
df21_22 G/SoT
df21_22 Save%
df22_23 G/SoT
df22_23 Save%


In [4]:
# Impute Missing Values
for df in dfx:
    for i in range(0, len(df)):
        if pd.isnull(df["G/SoT"][i]):
            if df["SoT%"][i] == 0:
                df["G/SoT"].iat[i] = 0
            else:
                df["G/SoT"].iat[i] = df["GF"][i]/(df["Shots"][i] * df["SoT%"][i])
        if pd.isnull(df["Save%"][i]):
            if df["SoT Against"][i] == 0:
                df["Save%"].iat[i] = 0
        if df["Save%"][i] < 0:
            df["Save%"].iat[i] = 0
        if df["Shots"][i] == 0:
            df["SoT%"].iat[i] = 0
            df["G/SoT"].iat[i] = 0
            df["Avg. Distance of Shots"].iat[i] = 0

# Check Again For NaN After Imputation
for df in dfx:
    for col in df.columns:
        if df[col].isnull().values.any():
            print(get_name(df), col)

Replacing Inconsistent Names

In [5]:
# Create Dictionary for Inconsistent Names
name_dict = {'Brighton':'Brighton and Hove Albion', 'Manchester Utd':'Manchester United', 'Newcastle Utd':'Newcastle United', 'Sheffield Utd':'Sheffield United', 
            "Nott'ham Forest":'Nottingham Forest', 'Tottenham':'Tottenham Hotspur', 'West Brom':'West Bromwich Albion', 'West Ham':'West Ham United', 'Wolves':'Wolverhampton Wanderers'}

# Replace Inconsistent Names in 'Opponent' & Add New Columns
for df in dfx:
    teams = list(dict.fromkeys(df["Team"]))
    exps = list(dict.fromkeys(df["Transfers - Home"]))
    df.insert(loc = 13, column = "Transfers - Away", value = ' ')
    season = get_name(df).replace('df', '')
    df["Season"] = season
    for i in range(0, len(df)):
        for key, value in name_dict.items():
            if df["Opponent"][i] == key:
                df["Opponent"].iat[i] = value
        for j in range(0, len(teams)):
            if df["Opponent"][i] == teams[j]:
                df["Transfers - Away"].iat[i] = exps[j]

Creating Moving Averages for Features

In [6]:
# Define a Function That Calculates a Moving Average of Three Previous Games & Creates New Columns
def moving_average(team_grouped, cols, new_cols):
    mov_avg = team_grouped[cols].rolling(3, closed = "left").mean()
    team_grouped[new_cols] = mov_avg
    team_grouped = team_grouped.dropna(subset = new_cols)
    return team_grouped

In [7]:
# Define the Input Columns
cols = df.columns[14:38]
cols = cols.append(df.columns[[6, 7, 10]])
new_cols = [f"{c} - Home" for c in cols]

# Group the Dataset by Team and Apply the MA Function
df_list = []
for df in dfx:
    df = df.groupby("Team").apply(lambda x: moving_average(x, cols, new_cols))
    df = df.droplevel("Team")
    df.index = range(df.shape[0])
    df["Match Number"] = range(1, len(df) + 1)
    df_list.append(df)

Adding Columns for Away Team

In [8]:
# Define Away Cols
away_cols = [col.replace('Home','Away') for col in new_cols]
away_cols.append('Match Number')

# Add Columns for Away Team
final_dfs = []
for df in df_list:
    away = []
    for i in range(0, len(df)):
        opp = df['Opponent'][i]
        mdate = df['Date'][i]
        team_df = df[df['Team'] == opp]
        nrow = team_df[team_df['Date'] == mdate][new_cols]
        nrow.insert(loc = 27, column = 'Match Number', value = i + 1)
        away.append(nrow)
    combine = pd.concat(away)
    combine.columns = away_cols
    combine.index = range(combine.shape[0])
    df = df.merge(combine, on = 'Match Number')
    final_dfs.append(df)

Merge Datasets & Encode Categorical Variables

In [9]:
# Merge Datasets
dfT = pd.concat(final_dfs)

# Filter 'Away' to Remove Duplicate Match
dfT = dfT[dfT['Venue'] == 'Home']

# Convert Categorical Data to Numerical & Fix Opponent Names
dfT["Date"] = pd.to_datetime(dfT["Date"], dayfirst = True)
dfT["Home Team"] = dfT["Team"].astype("category").cat.codes
dfT["Away Team"] = dfT["Opponent"].astype("category").cat.codes
dfT["Hour"] = dfT["Time"].str.replace(":.+", "", regex = True).astype("int")
dfT["Day of Week"] = dfT["Date"].dt.dayofweek
dfT["Month"] = dfT["Date"].dt.month
dfT["Output"] = (dfT["Result"] == 'W').astype("int")

In [10]:
# Drop Unwanted Features and Save to Excel
features = dfT.copy()
remove_cols = cols.append(dfT.columns[1:6]).append(dfT.columns[[11, 66]])
features = features.drop(columns = remove_cols)
features = features.sort_values("Date")
features.to_csv('Features.csv')